In [4]:
bronze_output = ""
silver_data = ""

StatementMeta(synonly, 16, 5, Finished, Available, Finished)

In [5]:
import json

# Parse the JSON string
bronze_data = json.loads(bronze_params)

# Access individual variables
start_date = bronze_data.get("start_date", "")
silver_adls = bronze_data.get("silver_adls", "")
gold_adls = bronze_data.get("gold_adls", "")

StatementMeta(synonly, 16, 6, Finished, Available, Finished)

' Data Factory\n\nimport json\n\n# Get base parameters\ndbutils.widgets.text("bronze_params", "")\ndbutils.widgets.text("silver_params", "")\n\nbronze_params = dbutils.widgets.get("bronze_params")\nsilver_params = dbutils.widgets.get("silver_params")\n\n# Debug: Print the raw input values for troubleshooting\nprint(f"Raw bronze_params: {bronze_params}")\nprint(f"Raw silver_params: {silver_params}")\n\n# Parse the JSON string\nbronze_data = json.loads(bronze_params)\n\n# Access individual variables\nstart_date = bronze_data.get("start_date", "")\nend_date = bronze_data.get("end_date", "")\nsilver_adls = bronze_data.get("silver_adls", "")\ngold_adls = bronze_data.get("gold_adls", "")\nsilver_data = silver_params\n\n# Debug: Print the extracted values for verification\nprint(f"Start Date: {start_date}, End Date: {end_date}")\nprint(f"Silver ADLS Path: {silver_adls}, Gold ADLS Path: {gold_adls}")\n'

In [6]:
# Ensure the below library is installed on your cluster
import reverse_geocoder as rg
from pyspark.sql.functions import col, udf, when
from pyspark.sql.types import StringType

StatementMeta(synonly, 16, 7, Finished, Available, Finished)

ModuleNotFoundError: No module named 'reverse_geocoder'

In [ ]:
df = spark.read.parquet(silver_data).filter(col("time") > start_date)

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
df = df.limit(10)  # added to speed up processings as during testing it was proving a bottleneck
# The problem is caused by the Python UDF (reverse_geocoder) being a bottleneck due to its non-parallel nature and high computational cost per task

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get("cc")
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
get_country_code(48.8588443, 2.2943506)

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
# adding country_code and city attributes
df_with_location = df.withColumn(
    "country_code", get_country_code_udf(col("latitude"), col("longitude"))
)

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
df.printSchema()

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
df_with_location.printSchema()

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
# adding significance classification
df_with_location_sig_class = df_with_location.withColumn(
    "sig_class",
    when(col("sig") < 100, "Low")
    .when((col("sig") >= 100) & (col("sig") < 500), "Moderate")
    .otherwise("High"),
)

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
df_with_location_sig_class.printSchema()

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
# Save the transformed DataFrame to the Silver container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
# Append DataFrame to Silver container in Parquet format
df_with_location_sig_class.write.mode("append").parquet(gold_output_path)

StatementMeta(, , , Cancelled, , Cancelled)